# GT2021-60283 Demo II- Create Train, Tune and Test Data Sets

In [1]:
import numpy as np
import sys
import pandas as pd
import pickle

In [2]:
label= 'upper_mic_theta_000_250_label_fut'
with pd.HDFStore("data/NTNU_LF.h5", 'r') as store:
    LF_df = store.get('LF0000').set_index('orig_idx', drop=False)
LF_df = LF_df.assign(**dict.fromkeys(["{}_{}".format('upper_mic_theta_000_H_250_label', i) for i in range(10)], 0))
for i in range(10):
    LF_df.iloc[:-((i+1)*25), -10+i]  = (LF_df['upper_mic_theta_000_H_250_label'].values[(i+1)*25:]).astype(float)

In [3]:
for i, key in enumerate(LF_df['key'].unique()):
    HF_file = 'data/{}_welch.h5'.format(key)
    with pd.HDFStore(HF_file, 'r') as store:
        temp_HF = store.get('T0000').set_index('orig_idx', drop=False)
    fft_cols = [col for col in temp_HF.keys() if col not in ['TIME', 'orig_idx']]
    if i ==0:
        LF_df = LF_df.assign(**dict.fromkeys(fft_cols, 0))
    LF_df.loc[temp_HF.index, fft_cols] = temp_HF[fft_cols]
    assert(np.all(LF_df.loc[temp_HF.index, 'TIME']==temp_HF['TIME']))

In [6]:
def add_fut(df, label):
    df_len = len(df)
    label_vals = np.zeros((int(10*df_len)))
    time_vals = np.zeros((int(10*df_len)))
    for i in range(10):
            label_vals[i*df_len:(i+1)*df_len] = df["{}_{}".format('upper_mic_theta_000_H_250_label', i)].values
            time_vals[i*df_len:(i+1)*df_len] = (i+1)*0.1

    df = pd.DataFrame(np.tile(df.values, (10, 1)), columns = df.keys())
    df[label] = label_vals
    df['TIME_fut'] = time_vals
    return df

In [11]:
train_tune_keys = ['ramp10_4kW', 'ramp17_4kW']
test_keys = ['ramp3_4kW']
train_tune_split = 0.7
train_df = LF_df[LF_df['key'].isin(train_tune_keys)].sample(frac=0.7)
tune_df = LF_df[LF_df['key'].isin(train_tune_keys)].drop(index=train_df.index)
test_df = LF_df[LF_df['key'].isin(test_keys)]
train_df = add_fut(train_df, label)
tune_df = add_fut(tune_df, label)
test_df = add_fut(test_df, label)

In [12]:
temp_keys = [key for key in LF_df.keys() if 'TC' in key or 'emperature' in key]
pred_combinations_orig = {'OP_T':['Z - Mass flow']+temp_keys,
                         'OP_FFT':['Z - Mass flow']+fft_cols,
                         'OP':['Z - Mass flow']}
pred_combinations = {}
for key, item in pred_combinations_orig.items():
    pred_combinations[key] = item
    pred_combinations[key] += ['TIME_fut']

In [ ]:
test_id = 'NTNU0000'
destination = 'data'
for key, item in pred_combinations.items():
    train = train_df[item+['orig_idx']].dropna()
    val = tune_df[item+['orig_idx']].dropna()
    test = test_df[item+['orig_idx']].dropna()
    train.to_csv("{}/{}_{}_train.csv".format(destination, test_id, key))
    val.to_csv("{}/{}_{}_tval.csv".format(destination, test_id, key))
    test.to_csv("{}/{}_{}_test.csv".format(destination, test_id, key))